In [ ]:
!pip install transformers datasets evaluate scikit-learn

In [ ]:
import pandas as pd
from transformers import ElectraTokenizer, ElectraForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
import evaluate
import ast
import random

# 1. 데이터 로드
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

# 2. 문제와 선택지를 결합하여 input_text 생성
failed_indices = []

def combine_question_and_choices(row):
    global failed_indices
    choices = row['선택지']

    # 선택지가 문자열로 저장된 경우 리스트로 변환 시도
    if isinstance(choices, str):
        try:
            choices = ast.literal_eval(choices)
        except (ValueError, SyntaxError):
            failed_indices.append(row.name)
            print(f"리스트 변환에 실패한 선택지: {choices}")
            return None  # 변환에 실패한 경우 None을 반환하여 나중에 삭제

    # 선택지가 리스트인 경우 정상적으로 결합
    if isinstance(choices, list) and len(choices) == 4:
        combined_text = row['문제'] + " " + " ".join(f"{i+1}. {choice}" for i, choice in enumerate(choices))
    else:
        failed_indices.append(row.name)
        combined_text = None

    return combined_text

# 'combine_question_and_choices' 함수를 적용
train_data['input_text'] = train_data.apply(combine_question_and_choices, axis=1)

if failed_indices:
    print(f"삭제할 행의 개수: {len(failed_indices)}")
    initial_size = train_data.shape[0]
    train_data = train_data.drop(failed_indices).reset_index(drop=True)
    deleted_count = initial_size - train_data.shape[0]
    print(f"삭제된 행의 개수: {deleted_count}")

print(f"변환 후 데이터셋 크기: {train_data.shape}")

tr_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

def preview_random_input_text(df, n=5):
    random_samples = df.sample(n=n, random_state=random.randint(1, 1000))
    for idx, row in random_samples.iterrows():
        print(f"Sample {idx}:\n{row['input_text']}\n")

print("tr_data에서 랜덤 샘플 미리보기:")
preview_random_input_text(tr_data, n=2)

print("val_data에서 랜덤 샘플 미리보기:")
preview_random_input_text(val_data, n=2)

model_name = "monologg/koelectra-base-v3-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=4)

from transformers import DataCollatorWithPadding

# DataCollatorWithPadding을 사용하여 동적으로 패딩 처리
data_collator = DataCollatorWithPadding(tokenizer, padding=True)

def tokenize_function(examples):
    # input_texts 리스트를 한 번에 처리
    inputs = tokenizer(
        examples["input_text"],  # 배치 전체를 처리
        padding="max_length",
        truncation=True,
        max_length=512,
    )

    # labels를 추가 (단일 정수 값이어야 함)
    labels = examples["답안"]

    # 입력과 레이블을 딕셔너리로 반환
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels
    }

# Train/Validation 데이터셋 토큰화
tr_dataset = Dataset.from_pandas(tr_data).map(tokenize_function, batched=True)
val_dataset = Dataset.from_pandas(val_data).map(tokenize_function, batched=True)

# Test 데이터 전처리
def preprocess_test_data(row):
    choices = row['선택지']

    if isinstance(choices, str):
        try:
            choices = ast.literal_eval(choices)
        except (ValueError, SyntaxError):
            print(f"리스트 변환에 실패한 선택지: {choices}")
            choices = ["기본 선택지1", "기본 선택지2", "기본 선택지3", "기본 선택지4"]

    if isinstance(choices, list) and len(choices) == 4:
        combined_text = row['문제'] + " " + " ".join(f"{i+1}. {choice}" for i, choice in enumerate(choices))
    else:
        combined_text = row['문제'] + " 1. 기본 선택지1 2. 기본 선택지2 3. 기본 선택지3 4. 기본 선택지4"

    return combined_text

test_data['input_text'] = test_data.apply(preprocess_test_data, axis=1)
test_dataset = Dataset.from_pandas(test_data).map(tokenize_function, batched=True)
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

tr_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

from transformers import EarlyStoppingCallback, TrainingArguments, Trainer

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # 매 에포크마다 평가
    learning_rate=2e-5,  # 학습률 조정
    per_device_train_batch_size=32,  # 배치 크기 조정
    per_device_eval_batch_size=32,  # 평가 배치 크기
    num_train_epochs=5,  # 에포크 수 조정
    weight_decay=0.01,  # 가중치 감쇠
    warmup_steps=200,  # 워밍업 단계 증가
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,  # Early Stopping을 위해 가장 좋은 모델을 로드
    save_total_limit=1,  # 저장하는 체크포인트 수 제한
    save_strategy="epoch",  # 매 에포크마다 저장
    metric_for_best_model="eval_loss",  # 최적 모델 선택 기준
    greater_is_better=False,  # loss가 낮을수록 좋음
)

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tr_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # 개선되지 않으면 2 에포크 후 중단
    data_collator=data_collator
)

# 모델 학습
trainer.train()

리스트 변환에 실패한 선택지: ['국가가 안정성을 제공하지 못함으로써 사회의 각 구성 요소가 자신의 안녕을 유지하기 위해 경쟁하게 되어 불안을 초래하는 상황을 말합니다. 이 조건은 자가 증식적이며, 체제를 확보하기 위한 조치가 더 큰 저항을 불러일으키기 때문에 반영구적인 긴급 무정부 상태가 됩니다.', '약소국의 불안정 딜레마는 주로 외부 조건으로 인해 발생하며, 구조적 무정부 상태와 유사한 상황을 초래합니다. 약소국이 자국의 지역적 입지를 개선하기 위한 조치를 취할 때 지역 내 불안을 조성합니다.', '약소국의 불안정 딜레마는 사회의 각 구성 요소가 자신의 안녕과 이익을 보호하고 유지하기 위한 경쟁에서 비롯됩니다. 그러나 지배 엘리트는 사회적 경쟁 영역과 분리되어 정책 딜레마를 초래합니다. 질서를 회복하기 위해 폭력 수단을 사용할 경우 체제의 기반이 약화됩니다.', '약소국의 불안정 딜레마는 정치적 및 제도적 중심화와 힘의 독점이 부족한 상황에서 발생합니다. 그러나 제도를 강화하기 위해 무력을 동원하면 이 과정을 중단시킬 수 있습니다. '국가성'을 육성하지 못하는 것은 폭력 사용으로 반전됩니다. 사회적 불안은 긴급하지만 개발되지 않은 무정부 상태의 반영구적 상황입니다.']
리스트 변환에 실패한 선택지: ['모든 RNA가 공통된 3' 말단에서 끝나고 중첩된 세트 전사체를 생성합니다', '긴 RNA 유전체와의 재조합을 이용합니다', '변이율이 높지 않습니다', '캡이 씌워진 세포 mRNA를 사용합니다']
리스트 변환에 실패한 선택지: ['정치적 편견 없이 진실되고 정확한 설명이어야 한다', '생존하지 못한 유사한 모든 문서를 대표해야 한다', '문자 그대로의 의미와 해석 가능한 의미를 모두 가져야 한다', '알려진 저자의 '신뢰할 수 있는' 원본 또는 신뢰할 수 있는 사본이어야 한다']
리스트 변환에 실패한 선택지: ['가난한 사람들이 상대적으로 박탈감을 느끼게 하는 문화적으로 가치 있는 상품과 생활 수준', '사람이 급여나 임금에서 받는 금전의 흐름', '토지

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

...

Map:   0%|          | 0/7194 [00:00<?, ? examples/s]

Map:   0%|          | 0/1799 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,1.386900,1.382421,0.288494
2,1.381100,1.379235,0.286826
3,1.354400,1.378672,0.287382
4,1.306700,1.394855,0.293496
5,1.264800,1.405725,0.282379


TrainOutput(global_step=1125, training_loss=1.3436122762891982, metrics={'train_runtime': 848.44, 'train_samples_per_second': 42.395, 'train_steps_per_second': 1.326, 'total_flos': 9464274610053120.0, 'train_loss': 1.3436122762891982, 'epoch': 5.0})

In [ ]:
# 모델 평가
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

Evaluation Results: {'eval_loss': 1.378671646118164, 'eval_accuracy': 0.2873818788215675, 'eval_runtime': 12.7507, 'eval_samples_per_second': 141.09, 'eval_steps_per_second': 4.47, 'epoch': 5.0}


In [ ]:
# 16. 답안 예측
predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(-1)

In [ ]:
# 17. 결과 저장
output = pd.DataFrame({
    "id": test_data["id"],
    "answer": predicted_labels
})

output.to_csv("submission.csv", index=False)